# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [5]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height==last_height:
        print('頁面已到達底端')
        break
    last_height=new_height
    print('更新中')

更新中
頁面已到達底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [6]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [7]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 廢文比產品文還好圈粉？年輕人創業「社交電商」搏感情心法大公開

▲2019年第三次「東森社交電商大會」除了集結來自東森全球新連鎖事業的經銷夥伴，同時也請來經營自媒體有成的年輕創業者到現場，和在座夥伴分享時下最in的「社交電商」概念。（圖／東森全球新連鎖事業提供，以下同）
記者萬玟伶／台中報導
2019年第三次「東森社交電商大會」12月28日（六）於台中金典酒店盛大登場，近千名會員擠滿現場，都是為了更加瞭解新興的「社交電商」而來！現場除了集結來自東森全球新連鎖事業的經銷夥伴，同時也請來經營自媒體有成的年輕創業者到現場，和在座夥伴分享時下最in的「社交電商」概念，並且帶來如何與粉絲建立良好互動、緊密連結的超強密技，將人脈變成錢脈！
此次大會特別邀請現為東森新聞雲記者，同時也是PetitV創意總監的黃稜涵分享她的「社交電商」密技，潮世代的網路語言與心理，大大刷新與會夥伴的認知！到底「社交電商」的第一步該如何開始？黃稜涵分享，由於她的銷售客群鎖定年輕人，因此選擇時下正夯的IG做為她的電商平台，「IG上有許多功能，可以幫使用者快速拉近與粉絲間的距離。像是限時動態、hashtag就是社交電商的行銷利器！」
在限時動態上，黃稜涵歸納出幾個重點，第一是「廢文比產品文還重要」，因為如果限時動態每一則都是產品文，粉絲一定很快往左滑，甚至完全噤聲、不回應。第二是「跟上流行時事」，當別人有、你沒有就落伍了，所以當人人都在媽祖托夢時，限時動態跟上話題，就能讓粉絲會心一笑。第三是利用「IG內建功能」，不論是投票、QA，都能夠快速跟粉絲互動，甚至瞭解粉絲想法。
▲152公分小隻女黃稜涵創立「PetitV」，專門為小隻女選擇適合衣服，並分享穿出黃金比例的撇步。
黃稜涵也分享近來她使用IG推廣自然美新品牌Angelita，「介紹產品特色時，讓皮膚變白、變嫩太基本了，我的切點就用仙女，粉絲一看到就會自己聯想到『仙體發光、水嫩亮白』。導購連結的稱呼也很重要，如果用『點這裡、看這裡』那大家真的是不會看，我下的是『想變仙女走這邊』，就能瞬間提升大家好奇，願意點進去。」
近來黃稜涵也在臉書FB成立「PV閨蜜交換日記」，跳脫自己是社交電商的身份，成為每一位粉絲的

▲ 台指期以台股作為交易標的。（示意圖／記者湯興漢攝）
記者李瑞瑾／台北報導
「臺股期貨指數」編製方法可參照股市採用發行股數作為加權依據，所以平日可以全數以近月臺指期作為指數代表成分計算指數，指數公司分析臺指期到期前之近月與次近月契約歷史未沖銷部位消長，在近月契約最後交易日前五天指數代表成分再加入次近月契約，並分五日依等比例將近月臺指期轉為次近月臺指期契約，表彰臺指期所有期別契約未平倉部位結構，計算指數評量臺指期之行情走勢。
▲ 回溯台股期貨指數績效表現分析。（表／記者李瑞瑾製）
----------------------------------------------------------------------

[8] 受尾牙旺季影響！　勞動部：住宿餐飲業求才數激增1萬5,380人

▲受尾牙旺季影響，住宿餐飲業求才數激增1萬5,380人。（圖／記者張一中攝）
記者余弦妙／台北報導
行政院主計總處日前公布國內108年11月失業率為3.73％，較108年10月下降0.04個百分點，較前一年同月上升0.03個百分點。根據勞動部勞動力發展署「台灣就業通」就業服務資料庫統計，「住宿及餐飲業」受到年底尾牙、聖誕及跨年餐飲旺季的加持，11月新登記求才人數達15,380人，較10月成長9.38％，成長人數居各行業之冠。
台灣就業通指出，108年11月各公立就業服務機構新登記求職人數總計54,099人，較10月減少8.31％，新登記求才人數為96,318人，較10月減少2.54％，平均每一位求職者有1.78個工作機會(即求供倍數為1.78倍)。
此外，台灣就業通也發現，隨著跨年、春節即將到來，與節慶商機相關的產業均逆勢成長。「批發及零售業」受惠於耶誕節及明年農曆春節鋪貨旺季來臨，帶動相關就業需求，11月新登記求才人數為13,402人，創下近3年同月新高紀錄，月增幅與年增幅分別達2.35％與8.03％。
在「批發及零售業」方面，近4成職缺集中在第一線的「商店銷售有關人員」，目前美廉社、家樂福、全聯實業、惠康百貨、愛買等量販超市均有大規模徵才。因應春節旺季，家樂福提早釋出上百位寒假工讀生職缺，工作性質涵蓋收銀人員、營運服務、購物車管理、網購營運等不同類型。
另外，在「住宿及餐飲業」方面，近5成職缺集中在第一線的「其他餐飲服務人員」，為搶攻耶誕、跨年商機，目前海底撈火鍋、雲雀國

▲勞動部。（圖／記者余弦妙攝）
記者余弦妙／台北報導
勞動部今（27）日表示，為保障具投票權的勞工得以行使參政權，勞動部已指定選舉投票日為「勞動基準法」所定放假日，因此，109年1月11日（星期六）2020總統大選投票當日，具有投票權且原屬工作日的勞工，雇主應依法讓勞工放假一日，工資照給。至於具投票權但當日原屬勞工的休息日或例假者，因勞工本得於當日行使投票權，因此不另外給假。
勞動部表示，放假「一日」指的是午前零時至午後12時連續24小時。由於投票權僅得於投票當日行使，性質與一般國定假日不同，因此，投票日不得再與其他工作日對調實施。
勞動部進一步說明，雇主如果徵得具投票權勞工同意於投票日出勤工作，應不妨礙其投票，並應加給當日出勤工資，至於計算方式分為下列兩種類型：
一、 原屬勞工的工作日：應加倍發給出勤時段的工資。舉例而言：勞雇雙方約定當天出勤8小時，勞工外出投票2小時，返回工作6小時，雇主須加給6小時的工資。
二、 原屬勞工的休息日：視工作時數依「勞動基準法」第24條第2項有關休息日出勤加給之規定計算。
勞動部條件司司長謝倩蒨舉例，一名勞工月薪3萬6000元，換算日薪為1200元、時薪150元，如在周六為工作日，投票日當天出勤6小時上班，則當日加班費應給予900元；若當日是休息日，雇主經徵得勞工同意後出勤6小時，加班費應給予1400元，至於部分工時工作者，則應給予2倍的時薪。
最後，勞動部再次提醒，雇主未依法給假或給薪者，可處新臺幣2萬元以上，最高100萬元罰鍰，並應補給工資。勞工權益如有受損害者，可就近向工作所在地勞工行政主管機關（即當地直轄市、縣市政府之勞工或社會局、處）申訴，以維權益。
